In [3]:
import cvxpy as cp
import numpy as np
import pandas as pd 

In [19]:
df = pd.read_csv('/home/fergus/data/mldatasets/energy_data/energydata_complete.csv')
y = df.Appliances.to_numpy()
df = df.drop(columns=['date', 'lights', 'Appliances', 'rv1', 'rv2'])
df

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,55.200000,...,18.2000,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000
1,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,55.200000,...,18.2000,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000
2,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,55.090000,...,18.2000,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000
3,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,55.090000,...,18.1000,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000
4,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,55.090000,...,18.1000,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,52.400000,...,24.7000,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333
19731,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,52.326667,...,24.7000,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000
19732,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,52.266667,...,24.7000,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667
19733,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,23.200000,52.200000,...,24.6625,49.518750,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333


In [21]:
A = df.to_numpy()
A.shape

(19735, 24)

First take the dataset and stack the isntances along the rows of a matrix. In the simple linear regression problem the output y is predicted by weighting each feature. Therefore the goal is to learn $ x $ so that $ y = Ax $. 

The easiest solution is $ y= A^{-1}y = A^{-1}Ax$. However this requires $A$ to be invertible - this is clearly not possible with a rectangular matrix

In [23]:
A.shape, y.shape

((19735, 24), (19735,))

The next solution is using ordinary least squares : 

$$ x= (A^T A)^{-1} A^T b$$

$A^TA$ is psd so invertible. 

To measure the fit can do use the l2 norm on the residual: $||Ax - y||_2$

In [26]:
x = np.linalg.inv(A.T @ A) @ A.T @ y

In [28]:
np.linalg.norm(A @ x - y, ord=2)

13306.4646469497

<br>
This ordinary least squares solution is the analytical solution of the optimisation problem: 

\begin{align*}
\text{minimize } & ||Ax-y||_2^2 
\end{align*}

To veryify this;



In [48]:
# ValueError: Incompatible dimensions (19735, 24) (19735, 1)

# Define inputs 
cpA = cp.Parameter((19735, 24))
cpA.value = A

cpY = cp.Parameter(19735)
cpY.value = y

cpX = cp.Variable(24)


obj = cp.Minimize(cp.pnorm(cpA @ cpX - cpY, 2) ** 2)
problem = cp.Problem(obj)
problem.solve()

inf

In [51]:
problem.value, cpX.value

(inf, None)

<br>

# Ridge Regression 

This extension of the least squares problem adds a penalty term to regularise the weight vector $x$.

\begin{align*}
\text{minimize } & ||Ax-y||_2^2 + \beta ||x||_2^2
\end{align*}


<br>

# Lasso Regression

This extension of the least squares problem uses the l1 norm to regularise $x$.

\begin{align*}
\text{minimize } & ||Ax-y||_2^2 + \beta ||x||_1
\end{align*}
